All the libraries, modules, imports and constants used in this program 

In [88]:
import pandas as pd
import mlxtend as ml

FILE_INPUT = "online_retail.csv"

This functions reads from the given file creating a dictionary of invoices:
<ol>
<li>skips the first row
<li>if the row starts with "C" is a cancellation -> to be skipped</li>
<li>split the row and convert it to integer</li>
<li>add the invoiceCode if not present to the data dictionary</li>
<li>if not present add the product code and description to the other dictionary</li>
<li>finally match invoice number with product number</li>
</ol>
Returns:
<ol>
<li>The dictionary #Invoice : set(#products)</li>
<li>The dictionary #products : description </li>
</ol>

In [89]:
def read_file(FILE_INPUT:str)->[dict[int:str], dict[int:str]]: # type: ignore
    invoices = dict()
    products = dict()
    
    with open(FILE_INPUT, "r", encoding='UTF-8') as file:
        file.readline()
        
        for row in file:    
            try:
                row = row.strip().split(",")
                row[0] = int(row[0])
                if len(row[1]) <= 1 or len(row[2]) <= 1:
                    raise ValueError 
            except ValueError:
                continue
            
            if row[0] not in invoices:
                invoices[row[0]] = set()
                
            if row[1] not in products:
                products[row[1]] = row[2]
                
            invoices[row[0]].add(row[1])
    
    return [invoices, products]

This functions creates a matrix starting from the products and the invoices:
- The rows are the invoices (which are sorted in ascending order)
- The columns are the products code (sorted in ascending order)
- The element (i, j) of the matrix is 1 if the product i is present in the invoice j, 0 otherwise

The matrix is then returned as a pd data frame

In [90]:
def create_matrix(invoices:dict[int:set[str]], invoices_id:list[int], products_id:list[str])->list[list[int]]:            
    return pd.DataFrame(data=[[1 if products_id[j] in invoices[invoices_id[i]] else 0 for j in range(len(products_id))] for i in range(len(invoices_id))],
                        columns= products_id, index=invoices_id) 

This is the main function of the program that will have all the main instructions and function calls of the whole program

In [91]:
def main()->None:
    [invoices, product] = read_file(FILE_INPUT)
    matrix = create_matrix(invoices, sorted(invoices.keys()), sorted(product.keys()))

    
main()

TypeError: Passing a set as an indexer is not supported. Use a list instead.